# സെമാന്റിക് കർണൽ OpenBnB MCP സെർവർ ഇന്റഗ്രേഷനോടൊപ്പം

ഈ നോട്ട്ബുക്ക് MCPStdioPlugin ഉപയോഗിച്ച് യഥാർത്ഥ OpenBnB MCP സെർവറുമായി സെമാന്റിക് കർണൽ ഉപയോഗിച്ച് യഥാർത്ഥ Airbnb താമസസ്ഥലങ്ങൾ തിരയുന്നത് എങ്ങനെ ചെയ്യാമെന്ന് കാണിക്കുന്നു. LLM ആക്സസിനായി, ഇത് Azure AI Foundry ഉപയോഗിക്കുന്നു. നിങ്ങളുടെ പരിസ്ഥിതി വേരിയബിളുകൾ സജ്ജമാക്കാൻ, നിങ്ങൾക്ക് [സജ്ജീകരണ പാഠം](/00-course-setup/README.md) പിന്തുടരാം.


## ആവശ്യമായ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP പ്ലഗിൻ കണക്ഷൻ സൃഷ്ടിക്കൽ

നാം MCPStdioPlugin ഉപയോഗിച്ച് [OpenBnB MCP സെർവർ](https://github.com/openbnb-org/mcp-server-airbnb) കണക്റ്റ് ചെയ്യും. ഈ സെർവർ @openbnb/mcp-server-airbnb പാക്കേജ് വഴി Airbnb തിരയൽ സൗകര്യം നൽകുന്നു.


## ക്ലയന്റ് സൃഷ്ടിക്കൽ

ഈ ഉദാഹരണത്തിൽ, LLM ആക്സസ് ചെയ്യാൻ Azure AI Foundry ഉപയോഗിക്കും. നിങ്ങളുടെ പരിസ്ഥിതി വേരിയബിളുകൾ ശരിയായി ക്രമീകരിച്ചിട്ടുണ്ടെന്ന് ഉറപ്പാക്കുക.


## പരിസ്ഥിതി ക്രമീകരണം

Azure OpenAI ക്രമീകരണങ്ങൾ ക്രമീകരിക്കുക. താഴെ പറയുന്ന പരിസ്ഥിതി വേരിയബിളുകൾ സജ്ജമാക്കിയിട്ടുണ്ടെന്ന് ഉറപ്പാക്കുക:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ഇന്റഗ്രേഷൻ മനസ്സിലാക്കുക

ഈ നോട്ട്ബുക്ക് **യഥാർത്ഥ OpenBnB MCP സെർവറുമായി** ബന്ധിപ്പിക്കുന്നു, ഇത് യഥാർത്ഥ Airbnb തിരച്ചിൽ പ്രവർത്തനക്ഷമത നൽകുന്നു.

### ഇത് എങ്ങനെ പ്രവർത്തിക്കുന്നു:

1. **MCPStdioPlugin**: MCP സെർവറുമായി സ്റ്റാൻഡേർഡ് ഇൻപുട്ട്/ഔട്ട്പുട്ട് ആശയവിനിമയം ഉപയോഗിക്കുന്നു  
2. **യഥാർത്ഥ NPM പാക്കേജ്**: `@openbnb/mcp-server-airbnb` എന്നത് npx വഴി ഡൗൺലോഡ് ചെയ്ത് പ്രവർത്തിപ്പിക്കുന്നു  
3. **തത്സമയ ഡാറ്റ**: അവരുടെ APIകളിൽ നിന്ന് യഥാർത്ഥ Airbnb പ്രോപ്പർട്ടി ഡാറ്റ തിരികെ നൽകുന്നു  
4. **ഫംഗ്ഷൻ കണ്ടെത്തൽ**: MCP സെർവറിൽ നിന്ന് ലഭ്യമായ ഫംഗ്ഷനുകൾ ഏജന്റ് സ്വയം കണ്ടെത്തുന്നു  

### ലഭ്യമായ ഫംഗ്ഷനുകൾ:

OpenBnB MCP സെർവർ സാധാരണയായി നൽകുന്നത്:  
- **search_listings** - സ്ഥലം, മാനദണ്ഡങ്ങൾ എന്നിവയുടെ അടിസ്ഥാനത്തിൽ Airbnb പ്രോപ്പർട്ടികൾ തിരയുക  
- **get_listing_details** - പ്രത്യേക പ്രോപ്പർട്ടികളുടെ വിശദമായ വിവരങ്ങൾ നേടുക  
- **check_availability** - പ്രത്യേക തീയതികൾക്കുള്ള ലഭ്യത പരിശോധിക്കുക  
- **get_reviews** - പ്രോപ്പർട്ടികൾക്കായുള്ള റിവ്യൂസ് നേടുക  
- **get_host_info** - പ്രോപ്പർട്ടി ഹോസ്റ്റുകളുടെ വിവരങ്ങൾ നേടുക  

### ആവശ്യമായ കാര്യങ്ങൾ:

- നിങ്ങളുടെ സിസ്റ്റത്തിൽ **Node.js** ഇൻസ്റ്റാൾ ചെയ്തിരിക്കണം  
- MCP സെർവർ പാക്കേജ് ഡൗൺലോഡ് ചെയ്യാൻ **ഇന്റർനെറ്റ് കണക്ഷൻ**  
- **NPX** ലഭ്യമായിരിക്കണം (ഇത് Node.js-നൊപ്പം വരുന്നു)  

### കണക്ഷൻ പരിശോധിക്കുക:

MCP സെർവർ മാനുവലായി പ്രവർത്തിപ്പിച്ച് നിങ്ങൾക്ക് പരിശോധിക്കാം:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
ഇത് OpenBnB MCP സെർവർ ഡൗൺലോഡ് ചെയ്ത് പ്രവർത്തിപ്പിക്കും, അതിനുശേഷം Semantic Kernel യഥാർത്ഥ Airbnb ഡാറ്റയ്ക്കായി അതുമായി ബന്ധപ്പെടും.


## OpenBnB MCP സെർവറുമായി ഏജന്റ് പ്രവർത്തിപ്പിക്കൽ

ഇപ്പോൾ, 2 മുതിർന്നവരും 1 കുട്ടിയും ഉള്ളവർക്കായി സ്റ്റോക്ക്ഹോം നഗരത്തിലെ യഥാർത്ഥ Airbnb താമസസ്ഥലങ്ങൾ തിരയാൻ OpenBnB MCP സെർവറുമായി ബന്ധപ്പെടുന്ന AI ഏജന്റ് പ്രവർത്തിപ്പിക്കാം. തിരയൽ മാനദണ്ഡങ്ങൾ മാറ്റാൻ `user_inputs` ലിസ്റ്റ് സ്വതന്ത്രമായി മാറ്റാം.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# സംഗ്രഹം
അഭിനന്ദനങ്ങൾ! Model Context Protocol (MCP) ഉപയോഗിച്ച് യഥാർത്ഥ ലോക താമസസൗകര്യങ്ങൾ തിരയുന്ന AI ഏജന്റ് നിങ്ങൾ വിജയകരമായി നിർമ്മിച്ചു:

## ഉപയോഗിച്ച സാങ്കേതികവിദ്യകൾ:
- Semantic Kernel - Azure OpenAI ഉപയോഗിച്ച് ബുദ്ധിമാനായ ഏജന്റുമാരെ നിർമ്മിക്കാൻ
- Azure AI Foundry - LLM ശേഷികൾക്കും ചാറ്റ് പൂർത്തീകരണത്തിനും
- MCP (Model Context Protocol) - സ്റ്റാൻഡേർഡൈസ്ഡ് ടൂൾ ഇന്റഗ്രേഷനായി
- OpenBnB MCP Server - യഥാർത്ഥ Airbnb തിരയൽ പ്രവർത്തനത്തിനായി
- Node.js/NPX - ബാഹ്യ MCP സെർവർ പ്രവർത്തിപ്പിക്കാൻ

## നിങ്ങൾ പഠിച്ചതെന്ത്:
- MCP ഇന്റഗ്രേഷൻ: Semantic Kernel ഏജന്റുമാരെ ബാഹ്യ MCP സെർവറുകളുമായി ബന്ധിപ്പിക്കൽ
- റിയൽ-ടൈം ഡാറ്റ ആക്സസ്: തത്സമയ APIകളിലൂടെ യഥാർത്ഥ Airbnb പ്രോപ്പർട്ടികൾ തിരയൽ
- പ്രോട്ടോക്കോൾ കമ്മ്യൂണിക്കേഷൻ: ഏജന്റും MCP സെർവറും തമ്മിലുള്ള stdio കമ്മ്യൂണിക്കേഷൻ ഉപയോഗിച്ച്
- ഫംഗ്ഷൻ ഡിസ്കവറി: MCP സെർവറുകളിൽ നിന്ന് ലഭ്യമായ ഫംഗ്ഷനുകൾ സ്വയം കണ്ടെത്തൽ
- സ്ട്രീമിംഗ് റെസ്പോൺസുകൾ: ഫംഗ്ഷൻ കോൾസ് തത്സമയത്തിൽ പകർത്തുകയും ലോഗ് ചെയ്യുകയും ചെയ്യൽ
- HTML റെൻഡറിംഗ്: ഏജന്റിന്റെ മറുപടികൾ സ്റ്റൈൽ ചെയ്ത ടേബിളുകളും ഇന്ററാക്ടീവ് ഡിസ്പ്ലേകളും ഉപയോഗിച്ച് ഫോർമാറ്റ് ചെയ്യൽ

## അടുത്ത ചുവടുകൾ:
- കൂടുതൽ MCP സെർവറുകൾ (കാലാവസ്ഥ, വിമാനങ്ങൾ, റസ്റ്റോറന്റുകൾ) ഇന്റഗ്രേറ്റ് ചെയ്യുക
- MCP, A2A പ്രോട്ടോക്കോളുകൾ സംയോജിപ്പിച്ച് ഒരു മൾട്ടി-ഏജന്റ് സിസ്റ്റം നിർമ്മിക്കുക
- നിങ്ങളുടെ സ്വന്തം ഡാറ്റാ സ്രോതസുകൾക്കായി കസ്റ്റം MCP സെർവറുകൾ സൃഷ്ടിക്കുക
- സെഷനുകൾക്കിടയിൽ സ്ഥിരമായ സംഭാഷണ മെമ്മറി നടപ്പിലാക്കുക
- MCP സെർവർ ഓർക്കസ്ട്രേഷനോടുകൂടി ഏജന്റിനെ Azure Functions-ലേക്ക് ഡിപ്ലോയ് ചെയ്യുക
- ഉപയോക്തൃ ഓതന്റിക്കേഷൻ, ബുക്കിംഗ് ശേഷികൾ ചേർക്കുക


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖ പ്രാമാണികമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
